# POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split 
from nltk.tokenize import word_tokenize
import time

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
nltk_data

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

In [3]:
# splitting data into training and test set
train_set, test_set = train_test_split(nltk_data,train_size=0.95,test_size=0.05,random_state=2112)

In [4]:
# list of tagged words in training set
train_tagged_words = [tw for sent in train_set for tw in sent]
len(train_tagged_words)

95802

In [5]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]


In [6]:
# vocabulary
voc = set(tokens)
print(len(voc))

12068


In [7]:
# Universal tag set 
tags = set([pair[1] for pair in train_tagged_words])
tags

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [8]:
test_tagged_words = [tw for sent in test_set for tw in sent]
len(test_tagged_words)
print(len(test_set))
print(len(train_set))

196
3718


## Build the vanilla Viterbi based POS tagger

### Viterbi based POS tagger (Model 1)

### Emission probabilities

In [9]:
t = len(tags)
v = len(voc)
w_given_t = np.zeros((t, v))

In [10]:
# Function to calculate emission probabilit: P(w\t)
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [11]:
# Function to calculate transition probability: P(t2|t1)

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [12]:
# creating t x t transition matrix of tags
# thus tags_matrix(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [13]:
# tags_matrix in pandas dataframe format
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [14]:
tags_df

,PRT,CONJ,.,NOUN,VERB,ADV,NUM,ADJ,X,PRON,DET,ADP
PRT,0.001965,0.002292,0.043549,0.246562,0.400786,0.010151,0.055010,0.085462,0.013425,0.018664,0.101179,0.020956
CONJ,0.005088,0.000463,0.035615,0.351989,0.154024,0.054117,0.039778,0.116559,0.008326,0.060130,0.119334,0.054579
.,0.002424,0.058618,0.093357,0.221993,0.088959,0.051975,0.081329,0.043896,0.026930,0.065799,0.173339,0.091293
NOUN,0.043538,0.042883,0.239425,0.264907,0.146997,0.017000,0.009428,0.012050,0.029086,0.004842,0.013178,0.176665
VERB,0.031664,0.004955,0.034760,0.110629,0.169080,0.081675,0.022606,0.065727,0.217233,0.035767,0.133545,0.092359
ADV,0.013671,0.007002,0.136045,0.032011,0.344115,0.078693,0.031344,0.131044,0.022674,0.015672,0.069356,0.118373
NUM,0.025908,0.013996,0.119714,0.347826,0.018166,0.002680,0.184932,0.033651,0.212031,0.001787,0.003276,0.036033
ADJ,0.011012,0.016601,0.065417,0.700033,0.011670,0.004767,0.020053,0.065417,0.021532,0.000493,0.005095,0.077909
X,0.184444,0.010317,0.163175,0.061905,0.206825,0.025079,0.002698,0.016508,0.073810,0.056508,0.054762,0.143968
PRON,0.012181,0.004949,0.040350,0.209745,0.483822,0.034640,0.007233,0.074229,0.093262,0.007613,0.008755,0.023220


In [15]:
# vanilla iterbi algorithm
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [16]:

# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# Removing tags from the words in above list
test_tagged_words = [tup[0] for sent in test_set for tup in sent]


### Evaluating tagging accuracy for vanilla Viterbi algorithm

In [17]:
# tagging the test sentences using Viterbi algorithm

start = time.time()
viterbi_tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [18]:
# Correctly predicted tags
correct_vit = [i for i, j in zip(viterbi_tagged_seq, test_run_base) if i == j] 

#Incorrectly predicted tags
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(viterbi_tagged_seq, test_run_base)) if j[0]!=j[1]]

In [19]:
print('Time taken',difference,'seconds')

Time taken 1205.8969328403473 seconds


In [20]:
accuracy = len(correct_vit)/len(viterbi_tagged_seq)
accuracy

0.9015182601559294

In [21]:
incorrect_tagged_cases

[[('battle', 'NOUN'), (('opens', 'PRT'), ('opens', 'VERB'))],
 [('opens', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))],
 [('0', 'X'), (('that', 'ADP'), ('that', 'DET'))],
 [(',', '.'), (('net', 'ADJ'), ('net', 'NOUN'))],
 [('$', '.'), (('306', 'PRT'), ('306', 'NUM'))],
 [('The', 'DET'), (('len', 'PRT'), ('len', 'NOUN'))],
 [('len', 'NOUN'), (("'s", 'VERB'), ("'s", 'PRT'))],
 [("'s", 'PRT'), (('foldability', 'PRT'), ('foldability', 'NOUN'))],
 [('be', 'VERB'), (('inserted', 'PRT'), ('inserted', 'VERB'))],
 [('smaller', 'ADJ'), (('incisions', 'PRT'), ('incisions', 'NOUN'))],
 [('eye', 'NOUN'), (('care', 'VERB'), ('care', 'NOUN'))],
 [('skin', 'NOUN'), (('care', 'VERB'), ('care', 'NOUN'))],
 [('-LRB-', '.'), (('Topix', 'PRT'), ('Topix', 'NOUN'))],
 [('gained', 'VERB'), (('16.05', 'PRT'), ('16.05', 'NUM'))],
 [('down', 'ADV'), (('1.46', 'PRT'), ('1.46', 'NUM'))],
 [('or', 'CONJ'), (('0.05', 'PRT'), ('0.05', 'NUM'))],
 [('at', 'ADP'), (('2691.19', 'PRT'), ('2691.19', 'NUM'))],
 [('international

### Solve the problem of unknown words

## Method1
### Let's combine rule based tagger with Viterbi tagger. 
#### The approach that I have used here is: _if emission probability of a word is 0 then tag it with rule based tagger else with Viterbi._

In [22]:
# Rules for Regex tagger
patterns = [
    (r'(The|the|A|a|An|an|This|That|this|that|Those|These|those|these)$', 'DET'), 
    (r'[A-z]*-[A-z]*','ADJ'), # Rule for instaces string-strin to tag as ADJ
    (r'.*ing$', 'VERB'),              # Words ending with -ing as verb
    (r'.*ed$', 'VERB'),               # Wrods ending with -ed as verb
    (r'.*ould$', 'VERB'),              # words ending with -ould as verb
    (r'.*able$', 'ADJ'),               # Words ending with -ing as verb
    (r'.*ly$', 'ADV'),              # words ending with -ly as ADV
    (r'(And|and|Or|or|But|but)$', 'CONJ'),
    (r'.*\'s$', 'NOUN'),              
    (r'.*s$', 'NOUN'),                
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),
    (r'(\*T\*)-[0-9]*', 'X'),         # Strings of format *T*-DIGITs as X 
    (r'.*', 'NOUN')                    
]

In [23]:
regex_tagger = nltk.RegexpTagger(patterns)

In [24]:
# Function to tag words using RegexTagger()
def rule_tagger(w):
    return(regex_tagger.tag(w))

In [25]:
#Viterbi algorithm combined with RegexTagger.
def Viterbi_mod2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        if pmax != 0:
            # if emission probability of word is not 0 then tag it using Viterbi.
            state_max = T[p.index(pmax)] 
            state.append(state_max)
        else:
            # if emission probability of word is 0 then tag it using regex tagger
            x=rule_tagger([word])
            state.append(x[0][1])

    return list(zip(words, state))

In [26]:
start_time2 = time.time()
viterbi2_tagged_sequence = Viterbi_mod2(test_tagged_words)
end_time2= time.time()
difference2 = end_time2 - start_time2

In [27]:
print('Time taken',difference,'seconds')

Time taken 1205.8969328403473 seconds


In [28]:
# Correctly predicted tags
correct_vit2 = [i for i, j in zip(viterbi2_tagged_sequence, test_run_base) if i == j] 
incorrect_vit2 = [i for i, j in zip(viterbi2_tagged_sequence, test_run_base) if i != j] 

#Incorrectly predicted tags
incorrect_tagged_cases2 = [[test_run_base[i-1],j] for i, j in enumerate(zip(viterbi2_tagged_sequence, test_run_base)) if j[0]!=j[1]]

In [29]:
accuracy_mod2 = len(correct_vit2)/len(viterbi2_tagged_sequence)
accuracy_mod2

0.9569142388182191

In [30]:
incorrect_tagged_cases2

[[('battle', 'NOUN'), (('opens', 'NOUN'), ('opens', 'VERB'))],
 [('0', 'X'), (('that', 'ADP'), ('that', 'DET'))],
 [(',', '.'), (('net', 'ADJ'), ('net', 'NOUN'))],
 [('eye', 'NOUN'), (('care', 'VERB'), ('care', 'NOUN'))],
 [('skin', 'NOUN'), (('care', 'VERB'), ('care', 'NOUN'))],
 [('blurred', 'VERB'), (('that', 'ADP'), ('that', 'DET'))],
 [('A', 'DET'), (('50-state', 'NOUN'), ('50-state', 'ADJ'))],
 [('also', 'ADV'), (('imposes', 'NOUN'), ('imposes', 'VERB'))],
 [('his', 'PRON'), (('choosing', 'VERB'), ('choosing', 'NOUN'))],
 [('to', 'PRT'), (('swing', 'NOUN'), ('swing', 'VERB'))],
 [('.', '.'), (('As', 'ADP'), ('As', 'ADV'))],
 [('.', '.'), (('Yet', 'CONJ'), ('Yet', 'ADV'))],
 [('his', 'PRON'), (('finest', 'NOUN'), ('finest', 'ADJ'))],
 [('so', 'ADV'), (('tight', 'ADJ'), ('tight', 'ADV'))],
 [("n't", 'ADV'), (('matter', 'NOUN'), ('matter', 'VERB'))],
 [('any', 'DET'), (('temporary', 'NOUN'), ('temporary', 'ADJ'))],
 [('with', 'ADP'), (('operating', 'NOUN'), ('operating', 'VERB'))],


## Method 2

## Let's combine Lexicon tagger and rule based tagger with Viterbi:

In [31]:
# BigramTagger backoff by UnigramTagger. UnigramTagger backoff by regex_tagger
unigram_tagger = nltk.UnigramTagger(train_set, backoff=  regex_tagger)
bigram_tagger= nltk.BigramTagger(train_set, backoff=  unigram_tagger)

bigram_tagger.evaluate(test_set)

0.9562987279441937

In [32]:
#Creating a function to tag a word using BigramTagger
def u_tag(w):
    return(bigram_tagger.tag(w))

In [33]:
def Viterbi_mod3(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        if pmax != 0:
        # getting state for which probability is maximum
        # if emission probability of word is not 0 then tag it using Viterbi.

            state_max = T[p.index(pmax)] 
            state.append(state_max)
        else:
        # if emission probability of word is  0 then tag it using BigramTagger

            y= u_tag([word])
            state.append(y[0][1])
    return list(zip(words, state))

In [34]:
start3= time.time()
viterbi3_seq= Viterbi_mod3(test_tagged_words)
end3= time.time()
difference3 = end3 - start3

In [35]:
print('time take',difference3,'seconds')

time take 1359.776432275772 seconds


In [36]:
# Correctly predicted tags
correct_rt3 = [i for i, j in zip(viterbi3_seq, test_run_base) if i == j] 

#Incorrectly predicted tags
incorrect_tagged_cases4 = [[test_run_base[i-1],j] for i, j in enumerate(zip(viterbi3_seq, test_run_base)) if j[0]!=j[1]]

In [37]:
accuracy_rt3 = len(correct_rt3)/len(viterbi3_seq)
accuracy_rt3

0.9569142388182191

In [38]:
incorrect_tagged_cases4

[[('battle', 'NOUN'), (('opens', 'NOUN'), ('opens', 'VERB'))],
 [('0', 'X'), (('that', 'ADP'), ('that', 'DET'))],
 [(',', '.'), (('net', 'ADJ'), ('net', 'NOUN'))],
 [('eye', 'NOUN'), (('care', 'VERB'), ('care', 'NOUN'))],
 [('skin', 'NOUN'), (('care', 'VERB'), ('care', 'NOUN'))],
 [('blurred', 'VERB'), (('that', 'ADP'), ('that', 'DET'))],
 [('A', 'DET'), (('50-state', 'NOUN'), ('50-state', 'ADJ'))],
 [('also', 'ADV'), (('imposes', 'NOUN'), ('imposes', 'VERB'))],
 [('his', 'PRON'), (('choosing', 'VERB'), ('choosing', 'NOUN'))],
 [('to', 'PRT'), (('swing', 'NOUN'), ('swing', 'VERB'))],
 [('.', '.'), (('As', 'ADP'), ('As', 'ADV'))],
 [('.', '.'), (('Yet', 'CONJ'), ('Yet', 'ADV'))],
 [('his', 'PRON'), (('finest', 'NOUN'), ('finest', 'ADJ'))],
 [('so', 'ADV'), (('tight', 'ADJ'), ('tight', 'ADV'))],
 [("n't", 'ADV'), (('matter', 'NOUN'), ('matter', 'VERB'))],
 [('any', 'DET'), (('temporary', 'NOUN'), ('temporary', 'ADJ'))],
 [('with', 'ADP'), (('operating', 'NOUN'), ('operating', 'VERB'))],


## Method 3
###  'NOUN' is the most frequently occuring tag in tree bank dataset So let's tag the words with 0 emission probability with tag 'NOUN'

In [39]:
def Viterbi_Noun(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        if pmax != 0:
        # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
            state.append(state_max)
        else:
        # if emission probability of a word is 0 then tag it as a 'NOUN'
            state.append('NOUN')
    return list(zip(words, state))

In [40]:
start4= time.time()
viterbi_noun_seq= Viterbi_Noun(test_tagged_words)
end4= time.time()
difference4 = end4 - start4

In [41]:
print('time take',difference4,'seconds')

time take 1440.2407121658325 seconds


In [42]:
# Correctly predicted tags
correct_noun = [i for i, j in zip(viterbi_noun_seq, test_run_base) if i == j] 

#Incorrectly predicted tags
incorrect_tagged_cases4 = [[test_run_base[i-1],j] for i, j in enumerate(zip(viterbi_noun_seq, test_run_base)) if j[0]!=j[1]]

In [43]:
accuracy_rt3 = len(correct_noun)/len(viterbi_noun_seq)
accuracy_rt3

0.9341403364792779

## Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm.
### Accuracy of vanilla Viterbi algorithm on nltk's tree bank dataset

In [44]:
correct_vit = [i for i, j in zip(viterbi_tagged_seq, test_run_base) if i == j] 
print(len(correct_vit)/len(viterbi_tagged_seq))

0.9015182601559294


### Accuracy of vanilla Viterbi algorithm combined with rule based algorithm on nltk's tree bank dataset

In [45]:
correct_vit2 = [i for i, j in zip(viterbi2_tagged_sequence, test_run_base) if i == j] 
print(len(correct_vit2)/len(viterbi2_tagged_sequence))

0.9569142388182191


### Accuracy of vanilla Viterbi algorithm combined with lexicon tagger and rule based algorithm on nltk's tree bank dataset

In [46]:
correct_rt3 = [i for i, j in zip(viterbi3_seq, test_run_base) if i == j] 
print(len(correct_rt3)/len(viterbi3_seq))

0.9569142388182191


### Accuracy of vanilla Viterbi algorithm combined with rule of tagging the word as 'NOUN' if emission probabillity of word is 0.

In [47]:
accuracy_rt3 = len(correct_noun)/len(viterbi_noun_seq)
accuracy_rt3

0.9341403364792779

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [48]:
test = pd.read_csv('Test_sentences.txt',header= None, error_bad_lines= False)

b'Skipping line 5: expected 1 fields, saw 2\nSkipping line 6: expected 1 fields, saw 2\n'


In [49]:
# untagged_words= 
test_sent= [sent for sent in test[0]]

In [50]:
test_sent

['Android is a mobile operating system developed by Google.',
 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.',
 "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.",
 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.',
 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.',
 'Show me the cheapest round trips from Dallas to Atlanta',
 'I would like to see flights from Denver to Philadelphia.',
 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.',
 'NASA invited social media users to experience the launch of ICESAT-2 Satellite.']

In [51]:
untagged_words = [word_tokenize(snt) for snt in test_sent]

In [52]:
untagged_words = [item for sublist in untagged_words for item in sublist]

In [53]:
untagged_words

['Android',
 'is',
 'a',
 'mobile',
 'operating',
 'system',
 'developed',
 'by',
 'Google',
 '.',
 'Android',
 'has',
 'been',
 'the',
 'best-selling',
 'OS',
 'worldwide',
 'on',
 'smartphones',
 'since',
 '2011',
 'and',
 'on',
 'tablets',
 'since',
 '2013',
 '.',
 'Google',
 'and',
 'Twitter',
 'made',
 'a',
 'deal',
 'in',
 '2015',
 'that',
 'gave',
 'Google',
 'access',
 'to',
 'Twitter',
 "'s",
 'firehose',
 '.',
 'Twitter',
 'is',
 'an',
 'online',
 'news',
 'and',
 'social',
 'networking',
 'service',
 'on',
 'which',
 'users',
 'post',
 'and',
 'interact',
 'with',
 'messages',
 'known',
 'as',
 'tweets',
 '.',
 'This',
 'is',
 'the',
 'first',
 'World',
 'Cup',
 'to',
 'be',
 'held',
 'in',
 'Eastern',
 'Europe',
 'and',
 'the',
 '11th',
 'time',
 'that',
 'it',
 'has',
 'been',
 'held',
 'in',
 'Europe',
 '.',
 'Show',
 'me',
 'the',
 'cheapest',
 'round',
 'trips',
 'from',
 'Dallas',
 'to',
 'Atlanta',
 'I',
 'would',
 'like',
 'to',
 'see',
 'flights',
 'from',
 'Denver'

In [54]:
len(untagged_words)

144

## Tagging words from given untagged test/sample sentences with vanilla Viterbi  algorithm

In [55]:
Viterbi_test= Viterbi(untagged_words)

In [56]:
Viterbi_test

[('Android', 'PRT'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'PRT'),
 ('.', '.'),
 ('Android', 'PRT'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'PRT'),
 ('worldwide', 'PRT'),
 ('on', 'ADP'),
 ('smartphones', 'PRT'),
 ('since', 'ADP'),
 ('2011', 'PRT'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'PRT'),
 ('.', '.'),
 ('Google', 'PRT'),
 ('and', 'CONJ'),
 ('Twitter', 'PRT'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'PRT'),
 ('that', 'DET'),
 ('gave', 'VERB'),
 ('Google', 'PRT'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'PRT'),
 ("'s", 'VERB'),
 ('firehose', 'PRT'),
 ('.', '.'),
 ('Twitter', 'PRT'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'PRT'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NO

### Tagging words from given untagged test/sample sentences with Method 1 (Viterbi algorithm combined with rule based tagger )

In [57]:
Viterbi_test_tag= Viterbi(untagged_words)
Viterbi_test_tag

[('Android', 'PRT'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'PRT'),
 ('.', '.'),
 ('Android', 'PRT'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'PRT'),
 ('worldwide', 'PRT'),
 ('on', 'ADP'),
 ('smartphones', 'PRT'),
 ('since', 'ADP'),
 ('2011', 'PRT'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'PRT'),
 ('.', '.'),
 ('Google', 'PRT'),
 ('and', 'CONJ'),
 ('Twitter', 'PRT'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'PRT'),
 ('that', 'DET'),
 ('gave', 'VERB'),
 ('Google', 'PRT'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'PRT'),
 ("'s", 'VERB'),
 ('firehose', 'PRT'),
 ('.', '.'),
 ('Twitter', 'PRT'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'PRT'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NO

In [58]:
Viterbi2_test_tag= Viterbi_mod2(untagged_words)

In [59]:
Viterbi2_test_tag

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NUM'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('s

### Tagging words from given untagged test/sample sentences with Method 1 (Viterbi algorithm combined with lexicon tagger backed-off by rule based tagger )

In [60]:
Viterbi3_test_tag= Viterbi_mod3(untagged_words)

In [61]:
Viterbi3_test_tag

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NUM'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('s

### Tagging words from given untagged test/sample sentences with Method 3 (Viterbi algorithm combined with 'NOUN' as a default tag for unknown words )

In [62]:
ViterbiNoun_test_tag= Viterbi_Noun(untagged_words)

In [63]:
ViterbiNoun_test_tag

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NOUN'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NOUN'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NOUN'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 

#### Main frame is dataframe consisting of words from untagged sentences from Test_Sentences.txt and corresponding predicted tags.
- _vanilla_Viterbi_tag_ column represents tags predicted by vanilla Viterbi algorithm.
- _Viterbi_regex_tag_ column represents tags predicted by Viterbi algorithm combined with rule based/ regex tagger.(Method 1)
- _Viterbi_bigram_tag_ column represents tags predicted by Viterbi algorith combined with bigram tagger.(Method 2)
- _Viterbi_Noun_tag_ column represents tags predicted by Viterbi algorith default tagger as 'NOUN' for words with 0 emission probability.(Method 3)

In [64]:
main_frame = pd.DataFrame()
main_frame['Words']= untagged_words
main_frame['vanilla_Viterbi_tag']= [i[1] for i in Viterbi_test]
main_frame['Viterbi_regex_tag'] = [i[1] for i in Viterbi2_test_tag]
main_frame['Viterbi_bigram_tag'] = [i[1] for i in Viterbi3_test_tag]
main_frame['Viterbi_Noun_tag']= [i[1] for i in ViterbiNoun_test_tag]
main_frame

,Words,vanilla_Viterbi_tag,Viterbi_regex_tag,Viterbi_bigram_tag,Viterbi_Noun_tag
0,Android,PRT,NOUN,NOUN,NOUN
1,is,VERB,VERB,VERB,VERB
2,a,DET,DET,DET,DET
3,mobile,ADJ,ADJ,ADJ,ADJ
4,operating,NOUN,NOUN,NOUN,NOUN
5,system,NOUN,NOUN,NOUN,NOUN
6,developed,VERB,VERB,VERB,VERB
7,by,ADP,ADP,ADP,ADP
8,Google,PRT,NOUN,NOUN,NOUN
9,.,.,.,.,.


In [65]:
# main_frame_filtered is the dataframe consisting of words which got different tag by vanilla_viterbi and iterbi_regex tagger
main_frame_filtered = main_frame[main_frame['vanilla_Viterbi_tag'] != main_frame['Viterbi_regex_tag']]
main_frame_filtered

,Words,vanilla_Viterbi_tag,Viterbi_regex_tag,Viterbi_bigram_tag,Viterbi_Noun_tag
0,Android,PRT,NOUN,NOUN,NOUN
8,Google,PRT,NOUN,NOUN,NOUN
10,Android,PRT,NOUN,NOUN,NOUN
15,OS,PRT,NOUN,NOUN,NOUN
16,worldwide,PRT,NOUN,NOUN,NOUN
18,smartphones,PRT,NOUN,NOUN,NOUN
20,2011,PRT,NUM,NUM,NOUN
25,2013,PRT,NUM,NUM,NOUN
27,Google,PRT,NOUN,NOUN,NOUN
29,Twitter,PRT,NOUN,NOUN,NOUN


# Summary
#### All the unknown words get tagged as first column(or row) name from tags_df when vanilla Viterbi algorithm implemented.
#### BigramTagger backoff by UnigramTagger and UnigramTagger backoff by rule based tagger combined with Viterbi algorithm   give exactly same accuracy as of  Viterbi tagger combined with rule based tagger .Because lexicon tagger backs off to rule based tagger when it encounters unknow word and all the words with emission probability 0 are unknown words.
#### Rule based tagger shows significant improvement in tagging accuracy.
#### Since most of the words in nltk's tree bank dataset are tagged 'NOUN', therefore Viterbi algorithm combined with the rule of tagging the unknown words with 'NOUN' shows significant increase in the tagging accuracy of vanilla Viterbi algorithm.
### Method 1, Method 2 and Method 3 managed to tag the unknown words such as Google, Android, Dallas, Philadelphia, Atlanta as 'NOUN' which is a correct tag.
### There are instaces such as 2015 2016 whic represent year hence a numerical quantity which got tagged incorrectly by vanilla viterbi tagger but got currectly tagged as 'NUM' by rule based tagger and lexicon  
### Because of the rule of tagging the words of format _string-string_, both method 1 and 2 managed to tag the word best-selling as ADJ.

### -  Columns Viterbi_regex_tag and Viterbi _bigram_tag are exactly same from main_frame dataframe and tags i nthese columns are correct with almost 95% accuracy. We can use these two columns as to compare the tags which were predicted incorrectly by vanilla Viterbi algorithm._
### - _Column Viterbi_Noun_tag has predicted valdation dataset tags with almost 93% accuracy.